# Video Reconstruction for Collective Activity Dataset (CAD-2)
**Author:** Christian Byron  **Date:** 19-Apr-21

This notebook provides an understanding of the Collective Activity dataset by reconstructing it into a video gui. 

The Collective Activity Dataset (available [here](http://vhosts.eecs.umich.edu/vision//activity-dataset.html)) was created by <sub>[1]</sub> 

- [ ] get the markup corrected to put the annotation description into a table 


- The original dataset contains 5 different collective activities : crossing, walking, waiting, talking, and queueing and 44 short video sequences some of which were recorded by consumer hand-held digital camera with varying view point.
- The augmented dataset (used here) added two more categories (dancing and jogging)

#### Annotation

Every 10th frame in all video sequences was annotated with image location of person, activity id, and pose direction.

Frame number, X, Y, WIDTH, HEIGHT, CLASS ID, POSE ID.
ex. 001       366     168     106     212     5       3
    001     512     190     98      195     5       3
    001     440     187     84      167     5       3
    001     339     191     83      165     5       3

CLASS ID 
1. NA, 2. Crossing, 3. Waiting, 4.Queuing, 5. Walking (not use), 6. Talking, 7. Dancing, 8. Jogging

POSE ID
1. Right 2. Front-right 3. Front 4. Front-left 5. Left 6. Back-left 7. Back 8. Back-right


### Step 1 - Find all videos and individual frames


In [1]:
import os
import re
import sys

base_video_collection_path = \
    "C:\\Users\\s441606\\OneDrive - University of Canberra\\2021 Semester 1\\z- Learning Python\\Jupyter\\videos"

video_folder_list = os.listdir(base_video_collection_path)
videos = {}

for video_folder in video_folder_list:

    video_starting_frame = sys.maxsize
    video_ending_frame = 0
    video_frames = {}
    video_annotations_all = {}
    video_annotations_options = list()
    
    video_annotations_file = open(base_video_collection_path + "\\" + video_folder + '\\annotations.txt', 'r')
    
    for video_annotations_line in video_annotations_file:
        video_annotation_line_search = re.search('\d+', video_annotations_line)
        if video_annotation_line_search: video_annotations_all.update({video_annotation_line_search.group(0): 
                                                                       video_annotations_line})
    
    
    for subdir, dirs, files in os.walk(base_video_collection_path + "\\" + video_folder):

        for file in files:
            if file.endswith(".jpg"):
                video_frame_number = os.path.splitext(file)[0]
                video_starting_frame = min(int(video_frame_number), video_starting_frame)
                video_ending_frame = max(int(video_frame_number), video_ending_frame)
                video_frames.update({int(video_frame_number):os.path.join(subdir, file)})
                
                if video_frame_number in video_annotations_all :
                    video_annotation_line_split = re.split('\s', video_annotations_all[video_frame_number])
                    video_annotation_line_split[0] = video_frame_number
                    video_annotations_options.append((video_frame_number + ' ' + video_annotation_line_split[1], 
                                                      video_annotation_line_split))
               
    video_obj = {"starting_frame":video_starting_frame, "ending_frame":video_ending_frame,"frames":video_frames,
                 "annotations_options":video_annotations_options} 
    
    videos.update({video_folder:video_obj})

#display(videos)

### Step 2 - Build the GUI Function to show video frames

In [2]:
import ipywidgets as widgets

imageArea = widgets.Image()

dropdown = widgets.Dropdown(
    options= video_folder_list,
    value ='1',
    description='Video #:',
    disabled=False,
)

play = widgets.Play(
        step=1,
        interval=50,
        description="Press play",
        disabled=False
)

slider = widgets.IntSlider()

widgets.jslink((play, 'value'), (slider, 'value'))

def on_dropdown_value_change(change): 
    video_selected = videos[change['new']]
    video_start = video_selected["starting_frame"]
    video_end = video_selected["ending_frame"]
    
    slider.min = play.min = 0
    slider.max = play.max = video_end
    slider.min = play.min = video_start
    slider.value = video_start
    
    annotations.options = video_selected['annotations_options']
  
  
dropdown.observe(on_dropdown_value_change, names='value')

def on_value_change(change):
    video_selected = videos[dropdown.value]
    
    file = open(f"{video_selected['frames'][change['new']]}","rb")
    imageArea.value = file.read()
 
slider.observe(on_value_change, names='value')

annotations = widgets.Dropdown(
    options= videos[dropdown.value]['annotations_options'],
    description='Annotated Frame #:',
    disabled=False,
)

controls = widgets.VBox([widgets.HBox([dropdown, play, slider,annotations]),imageArea])

# trigger the initial video to correctly set the slider values and load the first image
dropdown.value = '0'
controls

### Step 3 - Update the display when an Annotation is selected

In [3]:
import numpy as np
import cv2

def on_annotations_dropdown_value_change(change): 
    annotations_list = change['new']
    frame_number = int(annotations_list[0])
    play._playing = False
    slider.value = frame_number
    
    # convert the current frame image to a numpy array for opencv
    nparr = np.frombuffer(imageArea.value, np.uint8)
    im = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    #loop through each annotation set in the list 9 set of five values each
    for i in range(2,len( annotations_list) -5 ,5):
        start_point = (int(annotations_list[i]), int(annotations_list[i+1]))

        end_point = (int(annotations_list[i]) + int(annotations_list[i+2]), 
                     int(annotations_list[i+1]) + int(annotations_list[i+3]))
        
        im = cv2.rectangle(im, start_point, end_point, (255, 0, 0), 2)
        im = cv2.putText(im, annotations_list[i+4], start_point, cv2.FONT_HERSHEY_SIMPLEX , 
                         1, (255, 0, 0), 1)
        
   
    is_success, im_buf_arr = cv2.imencode(".jpg", im)
    byte_im = im_buf_arr.tobytes()

    imageArea.value = byte_im
    
    

annotations.observe(on_annotations_dropdown_value_change, names='value')

#### Lessons Learnt
- A lot of learning about the iPython widgets (inc lack of documentation on class interfaces). Especially on how to use observe method to select and run the video(cascading value changes)
- How to show a image using ipyWidgets - may be useful later - or may be replaced with other image libraries (eg Matlablib)
- A bit of a challenge to create the complex store of videos in a dictionary
- Folders are named after the annotated frame - not the beginning or end. So need to process differently to get the correct details
- Refresh syntax for regular expressions - used https://regex101.com/ as means to test and the best cheatsheet https://developer.mozilla.org/en-US/docs/Web/JavaScript/Guide/Regular_Expressions/Cheatsheet
- Started with openCV and use in jupyter - discovered natively cv2.imshow opens new windows :-P ; where as to open in Image widget needs conversion from bytes to numpy array (useful sites https://jdhao.github.io/2019/07/06/python_opencv_pil_image_to_bytes/ and https://intellipaat.com/community/15647/python-opencv-load-image-from-byte-string


#### References
[1]  Mostafa S. Ibrahim et al. “A Hierarchical Deep Temporal Model for Group Activity Recognition”. In:2016 IeeeConference on Computer Vision and Pattern Recognition.  IEEE  Conference  on  Computer  Vision  and  PatternRecognition.  2016,  pp.  1971–1980.isbn:  978-1-4673-8851-1